In [ ]:
import psycopg2

conn = psycopg2.connect(dbname="deeplabcut_db", user="postgres", password=1234)
cursor = conn.cursor()

query = "SELECT DISTINCT ON (health) health, coord_path FROM dlc_files WHERE name IS NOT NULL"
data = pd.read_sql_query(query, conn)

print("Unique names and corresponding coord_path:")
print(data)

conn.close()

In [ ]:
import psycopg2
import pandas as pd
import numpy as np

conn = psycopg2.connect(dbname="deeplabcut_db", user="postgres", password=1234)

query = "SELECT * FROM dlc_files ORDER BY ID LIMIT 1"
data = pd.read_sql_query(query, conn, index_col="id")
print(data)
conn.close()

# for id, row in data.iterrows():
#     print(f"id = {id}")
#     print(f"path = {row['normalized_path']}")
#     print('\n')

In [ ]:
data["normalized_path"].iloc[0]

In [ ]:
from calculate_total_distance_custom import calculate_total_distance_custom
total_distance, cumulative_distance = calculate_total_distance_custom(data["normalized_path"].iloc[1])
print(total_distance)

In [ ]:
query = "SELECT coord_path, name, maze, health, task FROM dlc_files WHERE task='food_only' AND health='Excitatory'"
# query = "SELECT coord_path FROM dlc_files WHERE task='food_only' AND health='Saline'"

data = pd.read_sql_query(query, conn)
# print(data)
for i in data.iterrows():
    print(i)

In [ ]:
import pandas as pd
import numpy as np
import os
import psycopg2

# --- Connect to database ---
conn = psycopg2.connect(dbname="deeplabcut_db", user="postgres", password=1234)

In [ ]:
import matplotlib.pyplot as plt

query = "SELECT distance FROM dlc_files"
df = pd.read_sql_query(query, conn)

# Step 2: Plot histogram
plt.figure(figsize=(8, 6))
plt.hist(df['distance'].dropna(), bins=30, edgecolor='black')
plt.xlabel('Distance')
plt.ylabel('Frequency')
plt.title('Histogram of Distances')
plt.show()

# Step 3: Plot boxplot
plt.figure(figsize=(6, 8))
plt.boxplot(df['distance'].dropna(), vert=True)
plt.ylabel('Distance')
plt.title('Boxplot of Distances')
plt.show()

In [ ]:
import importlib
import calculate_time_in_roi_20min

importlib.reload(calculate_time_in_roi_20min)
from calculate_time_in_roi_20min import calculate_time_in_roi_20min

In [ ]:
import psycopg2
import pandas as pd
import random

# Connect to your PostgreSQL database
conn = psycopg2.connect(
    dbname="deeplabcut_db",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)

# Fetch normalized paths and maze numbers
query = """
    SELECT normalized_path, maze
    FROM dlc_files
    WHERE normalized_path IS NOT NULL AND maze IS NOT NULL
"""
df = pd.read_sql_query(query, conn)
conn.close()

print(df.head())

In [ ]:
for i in range(5):
    maze = int(df["maze"][i])  # ← cast it to a native Python int
    file = df["normalized_path"][i]
    print(f"maze = {maze}, file = {file}")
    calculate_time_in_roi_20min(file, maze=maze, plot=True)

In [ ]:
updates = []
cursor.execute("""
    ALTER TABLE dlc_files ADD COLUMN IF NOT EXISTS distance FLOAT;
""")
cursor.execute("SELECT id, normalized_path FROM dlc_files")
rows = cursor.fetchall()

for row in rows:
    id_, path = row
    if not is_valid_csv(path, id_):
        continue
    try:
        total_distance = calculate_total_distance_20min(path)
    except Exception as e:
        print(f"Error for ID {id_}: {e}")
        continue

    if total_distance is not None:
        updates.append((total_distance, id_))
        print(f"Updated distance for ID {id_}: {total_distance:.2f}")
    else:
        print(f"Skipping ID {id_} — calculation failed.")

if updates:
    cursor.executemany("UPDATE dlc_files SET distance = %s WHERE id = %s", updates)

In [ ]:
import psycopg2
import pandas as pd
import numpy as np

conn = psycopg2.connect(dbname="deeplabcut_db", user="postgres", password=1234)

query = "SELECT * FROM dlc_files LIMIT 1"
data = pd.read_sql_query(query, conn)
print(data)

conn.close()

In [ ]:
import psycopg2

conn = psycopg2.connect(dbname="deeplabcut_db", user="postgres", password=1234)

query = "SELECT normalized_path FROM dlc_files  ORDER BY id LIMIT 5"
data = pd.read_sql_query(query, conn)
print(data)

In [ ]:
import importlib
import count_stopping_points

importlib.reload(count_stopping_points)
from count_stopping_points import count_stopping_points

In [ ]:
import pandas as pd
import psycopg2
from count_stopping_points import count_stopping_points

# Connect to the database
conn = psycopg2.connect(dbname="deeplabcut_db", user="postgres", password=1234)

# Query first 5 normalized file paths
query = "SELECT normalized_path FROM dlc_files ORDER BY id LIMIT 5"
data = pd.read_sql_query(query, conn)
conn.close()

# Loop through and plot
for i, row in data.iterrows():
    path = row['normalized_path']
    print(f"\n📂 Processing file {i+1}: {path}")
    count = count_stopping_points(path, plot=True)
    print(f"🛑 Stopping points: {count}")

In [ ]:
import pandas as pd
import psycopg2
from calculate_acc_jerk_outliers import calculate_acc_jerk_outliers

# Connect to the database
conn = psycopg2.connect(dbname="deeplabcut_db", user="postgres", password=1234)

# Query first 5 normalized file paths
query = "SELECT normalized_path FROM dlc_files ORDER BY id LIMIT 2"
data = pd.read_sql_query(query, conn)
conn.close()

# Loop through and plot
for i, row in data.iterrows():
    path = row['normalized_path']
    print(f"\nProcessing file {i+1}: {path}")
    acc_outliers, jerk_outliers = calculate_acc_jerk_outliers(path, plot=True)
    print(f"acc_outliers: {acc_outliers}, jerk_outliers: {jerk_outliers}")

In [ ]:
from update_outliers_to_db import update_outliers_to_db
update_outliers_to_db()

In [ ]:
from extract_treatment_ids import extract_treatment_ids
ids = extract_treatment_ids('food_only', 'Saline')

In [ ]:
import importlib
import plot_cumulative_distance

importlib.reload(plot_cumulative_distance)
from plot_cumulative_distance import plot_cumulative_distance

plot_cumulative_distance(ids)

In [ ]:
ids = extract_treatment_ids('food_only', 'Ghrelin')
plot_cumulative_distance(ids)

In [ ]:
ids = extract_treatment_ids('food_only', 'Inhibitory')
plot_cumulative_distance(ids)

In [ ]:
ids = extract_treatment_ids('food_only', 'Excitatory')
plot_cumulative_distance(ids)

In [ ]:
from extract_treatment_ids import extract_treatment_ids
from plot_grouped_cumulative_distance import plot_grouped_cumulative_distance

group_ids = {
    'Saline': extract_treatment_ids('food_only', 'Saline'),
    'Ghrelin': extract_treatment_ids('food_only', 'Ghrelin'),
    'Inhibitory': extract_treatment_ids('food_only', 'Inhibitory'),
    'Excitatory': extract_treatment_ids('food_only', 'Excitatory')
}

plot_grouped_cumulative_distance(group_ids)

In [ ]:
group_ids = {
    'Saline': extract_treatment_ids('light_only', 'Saline'),
    'Ghrelin': extract_treatment_ids('light_only', 'Ghrelin'),
    'Inhibitory': extract_treatment_ids('light_only', 'Inhibitory'),
    'Excitatory': extract_treatment_ids('light_only', 'Excitatory')
}

plot_grouped_cumulative_distance(group_ids)

In [ ]:
group_ids = {
    'Saline': extract_treatment_ids('toy_only', 'Saline'),
    'Ghrelin': extract_treatment_ids('toy_only', 'Ghrelin'),
    'Inhibitory': extract_treatment_ids('toy_only', 'Inhibitory'),
    'Excitatory': extract_treatment_ids('toy_only', 'Excitatory')
}

plot_grouped_cumulative_distance(group_ids)

In [ ]:
group_ids = {
    'Saline': extract_treatment_ids('food_light', 'Saline'),
    'Ghrelin': extract_treatment_ids('food_light', 'Ghrelin'),
    'Inhibitory': extract_treatment_ids('food_light', 'Inhibitory'),
    'Excitatory': extract_treatment_ids('food_light', 'Excitatory')
}

plot_grouped_cumulative_distance(group_ids)

In [ ]:
group_ids = {
    'Saline': extract_treatment_ids('toy_light', 'Saline'),
    'Ghrelin': extract_treatment_ids('toy_light', 'Ghrelin'),
    'Inhibitory': extract_treatment_ids('toy_light', 'Inhibitory'),
    'Excitatory': extract_treatment_ids('toy_light', 'Excitatory')
}

plot_grouped_cumulative_distance(group_ids)